In [29]:
#Import data/datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
#from nba_api.stats.static import players
#from nba_api.stats import endpoints
from sklearn import neighbors
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from scipy.stats import norm
from operator import itemgetter
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRFRegressor

In [30]:
historical_df = pd.read_csv('https://raw.githubusercontent.com/dribbleanalytics/ml-mvp-predict/master/2019-20-season/final-csv-data/historical-data.csv')

In [31]:
player_mvps = {}
player_listed = {}
previous_player = ['', 0]
num_won_mvps = []
num_listed_mvps = []
consecutive_mvps = []


for index, row in historical_df[:].iterrows():
  if row['player'] in player_listed.keys():
    player_listed[row['player']] += 1
    if row['player'] == previous_player[0]:
      consecutive_mvps.append(previous_player[1])
    else:
      consecutive_mvps.append(0)
    if row['won_mvp'] == 1:
      player_mvps[row['player']] += 1
      if previous_player[0] == row['player']:
        previous_player[1] += 1
      else:
        previous_player = [row['player'], 1]
  else:
    player_listed[row['player']] = 1
    player_mvps[row['player']] = 0
    consecutive_mvps.append(0)
    if row['won_mvp'] == 1:
      player_mvps[row['player']] += 1
      previous_player = [row['player'], 1]
    
  num_won_mvps.append(player_mvps[row['player']])
  num_listed_mvps.append(player_listed[row['player']])

consec = []
for i in range(0,349):
    consec.append(0)
consec[10] = 1
consec[22] = 2
consec[32] = 1
consec[41] = 1
consec[50] = 1
consec[62] = 2
consec[71] = 1
consec[83] = 2
consec[99] = 1
consec[104] = 1
consec[111] = 1
consec[121] = 1
consec[131] = 1
consec[153] = 1
consec[162] = 1
consec[178] = 1
consec[180] = 1
consec[191] = 2
consec[209] = 1
consec[219] = 2
consec[239] = 1
consec[248] = 1
consec[260] = 2
consec[279] = 1
consec[290] = 2
consec[308] = 1
consec[323] = 2
consec[332] = 1
consec[340] = 1

In [32]:
X = historical_df.copy()
X['num_mvps_won'] = num_won_mvps
X['num_top_rank'] = num_listed_mvps
X['consecutive_mvps'] = consec

X = X.drop(['rank', 'season_start', 'age', 'player', 'pos', 'pos_number', 'tm', 'pts_won', 'pts_max', 'all_star_votes', 'preseason_odds_rank', 'won_mvp'], axis=1)
Y = X['vote_share']

In [33]:
train, test = train_test_split(X, test_size=.2, random_state=42)

xtrain = train.copy();
ytrain = train['vote_share']
xtrain = xtrain.drop('vote_share', axis=1)

xtest = test.copy()
ytest = test['vote_share']
xtest = xtest.drop('vote_share', axis=1)

In [34]:
copy = X.copy()
copy['season_start'] = historical_df['season_start']
copy['player'] = historical_df['player']
y1984=copy[copy['season_start']==1984]
y1985=copy[copy['season_start']==1985]
y1986=copy[copy['season_start']==1986]
y1987=copy[copy['season_start']==1987]
y1988=copy[copy['season_start']==1988]
y1989=copy[copy['season_start']==1989]
y1990=copy[copy['season_start']==1990]
y1991=copy[copy['season_start']==1991]
y1992=copy[copy['season_start']==1992]
y1993=copy[copy['season_start']==1993]
y1994=copy[copy['season_start']==1994]
y1995=copy[copy['season_start']==1995]
y1996=copy[copy['season_start']==1996]
y1997=copy[copy['season_start']==1997]
y1998=copy[copy['season_start']==1998]
y1999=copy[copy['season_start']==1999]
y2000=copy[copy['season_start']==2000]
y2001=copy[copy['season_start']==2001]
y2002=copy[copy['season_start']==2002]
y2003=copy[copy['season_start']==2003]
y2004=copy[copy['season_start']==2004]
y2005=copy[copy['season_start']==2005]
y2006=copy[copy['season_start']==2006]
y2007=copy[copy['season_start']==2007]
y2008=copy[copy['season_start']==2008]
y2009=copy[copy['season_start']==2009]
y2010=copy[copy['season_start']==2010]
y2011=copy[copy['season_start']==2011]
y2012=copy[copy['season_start']==2012]
y2013=copy[copy['season_start']==2013]
y2014=copy[copy['season_start']==2014]
y2015=copy[copy['season_start']==2015]
y2016=copy[copy['season_start']==2016]
y2017=copy[copy['season_start']==2017]
y2018=copy[copy['season_start']==2018]

seasons=[y1984,y1985,y1986,y1987,y1988,y1989,y1990,y1991,y1992,y1993,y1994,y1995,y1996,y1997,y1998,y1999,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017,y2018]

In [35]:
xtrain_rf = xtrain.drop('num_mvps_won', axis = 1)
xtrain_rf = xtrain_rf.drop('num_top_rank', axis = 1)
xtrain_rf = xtrain_rf.drop('consecutive_mvps', axis = 1)

x_test_rf = xtest.drop('num_mvps_won', axis = 1)
x_test_rf = x_test_rf.drop('num_top_rank', axis = 1)
x_test_rf = x_test_rf.drop('consecutive_mvps', axis = 1)

RF_model = XGBRFRegressor(n_estimators=55, learning_rate=1.1, max_depth=5, colsample_bynode=0.69, subsample=0.71)

RF_model.fit(xtrain_rf, ytrain.values.ravel())
y_pred_rf = RF_model.predict(x_test_rf)
print("Mean squared error: %.3f" % mean_squared_error(ytest, y_pred_rf))
print('R2 score: %.3f' % r2_score(ytest, y_pred_rf))

[00:23:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean squared error: 0.033
R2 score: 0.447


In [36]:
param_grid = {
    "n_estimators": [10, 25, 50, 100, 250, 500, 1000, 5000],
    "max_depth": [5, 10, 15, 20],
    "learning_rate": [0.9, 1, 1.1],
    "subsample": [0.5, 0.6, 0.7, 0.8, 0.9],
    "colsample_bynode": [0.5, 0.6, 0.7, 0.8, 0.9]
}


# model_cv = XGBRFRegressor()
# grid_cv = GridSearchCV(model_cv, param_grid, n_jobs=-1, cv=3, scoring="r2")
# grid_cv.fit(xtrain, ytrain.values.ravel())
# y_pred_cv = grid_cv.predict(xtest)
# print(grid_cv.best_params_)
# print("CV Mean squared error: %.3f" % mean_squared_error(ytest, y_pred_cv))
# print('CV R2 score: %.3f' % r2_score(ytest, y_pred_cv))

In [37]:
all_time_preds = []

for season in seasons:
  xRF = season.drop(['num_mvps_won', 'num_top_rank', 'consecutive_mvps', 'player', 'vote_share', 'season_start'], axis=1)
  RF_pred = RF_model.predict(xRF)
  season_pred = pd.DataFrame({"Year": season['season_start'],
                      "Player": season['player'],
                      "Prediction": RF_pred })
  season_pred = season_pred[season_pred['Prediction']>= 0]
  season_pred = season_pred.sort_values('Prediction', ascending=False)
  all_time_preds.append(season_pred['Player'].iloc[0])
#print(all_time_preds)

true= pd.DataFrame({"Year": historical_df['season_start'],
                     "MVP": historical_df['player'],
                     "i": historical_df['won_mvp']})
true = true[true['i'] == 1]
true = true.drop(['i'], axis=1)
true['RF_Predicted'] = all_time_preds

def highlight_col(x):
    #copy df to new - original data are not changed
    df = x.copy()
    #set default values to all values
    df.loc[:,:] = 'background-color: ""'
    #set by condition
    for i in range(35):
      if x.iloc[i,2]!=x.iloc[i,1]:
          df.iloc[i,2] = 'color: red'
      else:
          df.iloc[i,2] = 'color: black'
    return df    

true.style.apply(highlight_col, axis=None)

,Year,MVP,RF_Predicted
0,1984,Larry Bird,Larry Bird
10,1985,Larry Bird,Larry Bird
20,1986,Magic Johnson,Magic Johnson
30,1987,Michael Jordan,Larry Bird
40,1988,Magic Johnson,Magic Johnson
50,1989,Magic Johnson,Magic Johnson
61,1990,Michael Jordan,Michael Jordan
71,1991,Michael Jordan,Michael Jordan
81,1992,Charles Barkley,Charles Barkley
90,1993,Hakeem Olajuwon,Hakeem Olajuwon


In [38]:
cop = X.drop(['vote_share', 'trb_perc'], axis=1)
RF_model.fit(cop,Y.values.ravel())

[00:23:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRFRegressor(colsample_bynode=0.69, learning_rate=1.1, max_depth=5,
               n_estimators=55, subsample=0.71)

In [48]:
prediction_set = {
  'player':['Stephen Curry', 'Kevin Durant', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Chris Paul', 'Jimmy Butler', 'Rudy Gobert', 'James Harden', 'Deandre Ayton', 'DeMar DeRozan'],
  'age':[33, 33, 27, 26, 36, 32, 29, 32, 23, 32],
  'g':[22, 22, 21, 17, 23, 17, 22, 23, 17, 24],
  'team_wins':[19, 16, 15, 11, 19, 14, 16, 16, 19, 16],
  'overall_seed':[1, 3, 6, 18, 2, 7, 4, 3, 2, 4],
  'mp':[34.4, 36.0, 32.7, 32.3, 32.3, 34.2, 31.6, 35.7, 30.6, 35.3],
  'fgm':[9, 10.4, 9.8, 10.2, 5.0, 7.9, 5.4, 5.9, 7.4, 9.5],
  'fga':[20.6, 19.5, 18.4, 17.4, 10.5, 15.3, 7.3, 14.6, 11.9, 19.0],
  'fg_perc':[0.437, 0.533, 0.532, 0.590, 0.479, 0.519, 0.739, 0.403, 0.619, 0.498],
  '3pm':[5.4, 1.8, 1.1, 1.7, 1.0, 0.4, 0, 2.5, 0.1, 0.8],
  '3pa':[13.2, 4.6, 4.1, 4.4, 3.0, 1.8, 1, 7.3, 0.3, 2.5],
  '3p_perc':[0.41, 0.382, 0.276, 0.387, 0.338, 0.200, 0, 0.347, 0.200, 0.333],
  '2pm':[3.6, 8.6, 8.7, 8.5, 4.0, 7.6, 5.4, 3.3, 7.3, 8.6],
  '2pa':[7.4, 14.8, 14.3, 12.9, 7.6, 13.5, 7.3, 7.3, 11.6, 16.5],
  '2p_perc':[0.485, 0.580, 0.607, 0.659, 0.534, 0.561, 0.744, 0.458, 0.629, 0.523],
  'efg':[0.568, 0.578, 0.563, 0.573, 0.522, 0.531, 0.739, 0.490, 0.621, 0.520],
  'ftm':[4.1, 6.1, 6.9, 3.9, 3.2, 7.4, 4.5, 6.3, 1.7, 6.6],
  'fta':[4.5, 7.0, 10.2, 5.1, 3.8, 8.6, 6.6, 7.2, 2.8, 7.5],
  'ft_perc':[0.929, 0.870, 0.673, 0.759, 0.841, 0.850, 0.676, 0.885, 0.617, 0.888],
  'orb':[0.6, 0.5, 1.9, 2.6, 0.3, 1.6, 2.9, 1.0, 3.0, 0.6],
  'drb':[5.1, 7.3, 10.0, 7.3, 3.7, 4.2, 11.5, 6.9, 8.2, 4.7],
  'trb':[5.7, 7.7, 11.8, 9.9, 4.0, 5.8, 14.4, 7.9, 11.2, 5.3],
  'ast':[6.5, 5.5, 6.0, 6.0, 10.1, 5.3, 0.9, 9.5, 1.4, 4.1],
  'stl':[1.8, 0.6, 1.1, 1.1, 2.0, 2.1, 1.0, 1.5, 0.8, 0.9],
  'blk':[0.5, 0.7, 1.7, 0.7, 0.3, 0.3, 2.1, 0.7, 0.7, 0.4],
  'tov':[3.1, 3.0, 3.2, 2.6, 2.4, 1.9, 2.0, 4.8, 1.6, 2.0],
  'pf':[1.8, 1.6, 2.9, 2.8, 2.0, 1.7, 2.7, 2.4, 2.4, 2.3],
  'pts':[27.5, 28.6, 27.6, 26.1, 14.3, 23.6, 15.3, 20.6, 16.5, 26.4],
  'per':[25.4, 26.1, 31.5, 34.5, 21.4, 27.8, 25.1, 20.7, 21.5, 23.6],
  'ts':[0.611, 0.634, 0.603, 0.665, 0.586, 0.618, 0.747, 0.581, 0.629, 0.592],
  '3par':[0.640, 0.238, 0.225, 0.254, 0.281, 0.115, 0.006, 0.499, 0.025, 0.592],
  'ftr':[0.216, 0.360, 0.553, 0.295, 0.364, 0.565, 0.901, 0.493, 0.233, 0.393],
  'orb_perc':[1.8, 1.4, 6.0, 9.1, 0.9, 5.3, 10.7, 3.1, 10.6, 2.0],
  'drb_perc':[13.6, 20.4, 30.7, 37.7, 11.8, 13.7, 37.8, 19.5, 27.1, 14.4],
  'ast_perc':[29.5, 28.1, 34.4, 38.4, 43.4, 26.7, 4.2, 11.6, 6.9, 19.7],
  'stl_perc':[2.5, 0.9, 1.6, 1.7, 3.0, 3.1, 1.5, 2.0, 1.3, 1.3],
  'blk_perc':[1.4, 1.7, 5.0, 2.2, 0.7, 1.0, 5.7, 1.7, 2.0, 0.9],
  'tov_perv':[12.1, 11.7, 12.2, 14.2, 16.4, 9.2, 16.4, 21.4, 10.8, 8.4],
  'usg_perc':[30.3, 31.2, 34.2, 31.9, 19.6, 27.2, 16.8, 27.8, 20.8, 30.3],
  'ws':[4.0, 3.7, 4.1, 3.6, 3.4, 3.4, 4.0, 2.7, 2.2, 3.5],
  'ws_per_48':[0.255, 0.225, 0.288, 0.316, 0.221, 0.284, 0.276, 0.158, 0.200, 0.200],
  'vorp':[2.0, 1.8, 2.2, 2.3, 1.2, 1.6, 1.1, 1.2, 0.5, 1.1],
  'bpm':[8.6, 6.9, 10.5, 14.1, 4.1, 8.8, 4.5, 3.7, 1.5, 2.9],
  'offensive_raptor':[7.2, 4.4, 4.8, 7.3, 3.8, 3.5, -0.5, 5.2, 0.3, 1.9],
  'defensive_raptor':[0, -0.5, 3.2, 2.7, -1.0, 1.9, 6.3, -0.8, 1.0, -1.4],
  'num_mvps_won':[2, 1, 2, 1, 0, 0, 0, 1, 0, 0],
  'num_top_rank':[8, 10, 6, 4, 9, 2, 2, 9, 0, 2],
  'consecutive_mvps':[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

}

In [49]:
predictions_names = pd.DataFrame(prediction_set['player'])
prediction_set = pd.DataFrame(prediction_set).drop(['age', 'player'], axis=1)
results = RF_model.predict(prediction_set)

prediction = pd.DataFrame({"Player": predictions_names[0],
                           'RF*_Votes': results})

prediction = prediction.sort_values('RF*_Votes', ascending=False)
prediction

,Player,RF*_Votes
3,Nikola Jokic,0.296300
8,Deandre Ayton,0.282692
6,Rudy Gobert,0.260178
0,Stephen Curry,0.223727
2,Giannis Antetokounmpo,0.218094
1,Kevin Durant,0.154151
5,Jimmy Butler,0.134973
7,James Harden,0.124047
4,Chris Paul,0.099612
9,DeMar DeRozan,0.067591
